In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import csv
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

np.random.seed(42)
import random
random.seed(42)
X_dicts = []
y = []

with open('/kaggle/input/health-insurance-cross-sell-prediction/train.csv') as in_file:
    iCSV = csv.reader(in_file, delimiter=',')
    header = next(iCSV)
    for row in iCSV:
        new_dict={}        
        new_dict[header[1]] = row[1]
        new_dict[header[6]] = row[6]
        new_dict[header[7]] = row[7]
        for h, i in zip(header[2:6], row[2:6]):
            new_dict[h] = float(i)
        for j, k in zip(header[8:-1], row[8:-1]):
            new_dict[j] = float(k)
        X_dicts.append(new_dict) 
        
        y.append(int(row[-1]))
        
in_file.close()


vec = DictVectorizer(sparse=False) 

X = vec.fit_transform(X_dicts)
y = np.array(y)


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV

params = {'n_estimators':[10, 100, 200, 500]}

clf = GradientBoostingClassifier(random_state=42)

clf_gb = GridSearchCV(clf, params, scoring='f1_micro', cv = 10, n_jobs = 3) 

clf_gb.fit(X, y) 
print(clf_gb.best_score_)

print(clf_gb.best_params_) 


In [ ]:
idList = list()
X_dicts_test = list()

with open('/kaggle/input/health-insurance-cross-sell-prediction/test.csv') as in_file:
    iCSV = csv.reader(in_file, delimiter=',')
    header = next(iCSV)
    for row in iCSV:
        
        idList.append(row[0])
        new_dict={}        
        new_dict[header[1]] = row[1]
        new_dict[header[6]] = row[6]
        new_dict[header[7]] = row[7]
        for h, i in zip(header[2:6], row[2:6]):
            new_dict[h] = float(i)
        for j, k in zip(header[8:-1], row[8:-1]):
            new_dict[j] = float(k)
        X_dicts_test.append(new_dict) 
        
in_file.close()

X_new = vec.transform(X_dicts_test)


In [ ]:
preds = clf_gb.predict_proba(X_new)[:, 1]
preds[preds >= 0.5] = 1
preds[preds < 0.5] = 0


print(len([X for x in preds if x == 0]))
print(len([X for x in preds if x == 1]))

In [ ]:
import csv
fields = ['id', 'Response']
filename = 'Customer response.csv'

outputList = list()

for i in range(0,len(preds)):
    finalList = list()
    finalList.append(idList[i])
    finalList.append(int(preds[i]))
    outputList.append(finalList)

with open(filename, 'wt') as f:
    csv_writer = csv.writer(f)

    csv_writer.writerow(fields) # write header

    for row in outputList:
        csv_writer.writerow(row)